In [8]:
import matplotlib.pyplot as plt

In [1]:
from config import get_arguments
from SinGAN.manipulate import *
from SinGAN.training import *
from SinGAN.imresize import imresize
from SinGAN.imresize import imresize_to_shape
import SinGAN.functions as functions

In [2]:
parser = get_arguments()
parser.add_argument('--input_dir', help='input image dir', default='Input/Images')
parser.add_argument('--input_name', help='training image name', required=False, default="dot_mask_new_3ch.png") # added default by vajira
parser.add_argument('--ref_dir', help='input reference dir', default='Input/Paint')
parser.add_argument('--ref_name', help='reference image name', required=False, default="sperm_6_3ch_gray.png")
parser.add_argument('--paint_start_scale', help='paint injection scale', type=int, required=False, default=1) # added default by vajira
parser.add_argument('--quantization_flag', help='specify if to perform color quantization training', type=bool, default=False)
parser.add_argument('--mode', help='task to be done', default='paint2image')
opt = parser.parse_args("")

In [3]:
opt

Namespace(Dsteps=3, Gsteps=3, alpha=10, beta1=0.5, gamma=0.1, input_dir='Input/Images', input_name='dot_mask_new_3ch.png', ker_size=3, lambda_grad=0.1, lr_d=0.0005, lr_g=0.0005, manualSeed=None, max_size=250, min_nfc=32, min_size=25, mode='paint2image', nc_im=3, nc_z=3, netD='', netG='', nfc=32, niter=2000, noise_amp=0.1, not_cuda=0, num_layer=5, out='Output', padd_size=0, paint_start_scale=1, quantization_flag=False, ref_dir='Input/Paint', ref_name='sperm_6_3ch_gray.png', scale_factor=0.75, stride=1)

In [4]:
opt = functions.post_config(opt)
Gs = []
Zs = []
reals = []
NoiseAmp = []
dir2save = functions.generate_dir2save(opt)

Random Seed:  5792


In [6]:
if dir2save is None:
        print('task does not exist')
    #elif (os.path.exists(dir2save)):
    #    print("output already exist")
else:
    try:
        os.makedirs(dir2save)
    except OSError:
        pass
    real = functions.read_image(opt)
    real = functions.adjust_scales2image(real, opt)
    Gs, Zs, reals, NoiseAmp = functions.load_trained_pyramid(opt)
    if (opt.paint_start_scale < 1) | (opt.paint_start_scale > (len(Gs)-1)):
        print("injection scale should be between 1 and %d" % (len(Gs)-1))
    else:
        ref = functions.read_image_dir('%s/%s' % (opt.ref_dir, opt.ref_name), opt)
        if ref.shape[3] != real.shape[3]:
            ref = imresize_to_shape(ref, [real.shape[2], real.shape[3]], opt)
            ref = ref[:, :, :real.shape[2], :real.shape[3]]

        N = len(reals) - 1
        n = opt.paint_start_scale
        in_s = imresize(ref, pow(opt.scale_factor, (N - n + 1)), opt)
        in_s = in_s[:, :, :reals[n - 1].shape[2], :reals[n - 1].shape[3]]
        in_s = imresize(in_s, 1 / opt.scale_factor, opt)
        in_s = in_s[:, :, :reals[n].shape[2], :reals[n].shape[3]]
        if opt.quantization_flag:
            opt.mode = 'paint_train'
            dir2trained_model = functions.generate_dir2save(opt)
            # N = len(reals) - 1
            # n = opt.paint_start_scale
            real_s = imresize(real, pow(opt.scale_factor, (N - n)), opt)
            real_s = real_s[:, :, :reals[n].shape[2], :reals[n].shape[3]]
            real_quant, centers = functions.quant(real_s, opt.device)
            plt.imsave('%s/real_quant.png' % dir2save, functions.convert_image_np(real_quant), vmin=0, vmax=1)
            plt.imsave('%s/in_paint.png' % dir2save, functions.convert_image_np(in_s), vmin=0, vmax=1)
            in_s = functions.quant2centers(ref, centers)
            in_s = imresize(in_s, pow(opt.scale_factor, (N - n)), opt)
            # in_s = in_s[:, :, :reals[n - 1].shape[2], :reals[n - 1].shape[3]]
            # in_s = imresize(in_s, 1 / opt.scale_factor, opt)
            in_s = in_s[:, :, :reals[n].shape[2], :reals[n].shape[3]]
            plt.imsave('%s/in_paint_quant.png' % dir2save, functions.convert_image_np(in_s), vmin=0, vmax=1)
            if (os.path.exists(dir2trained_model)):
                # print('Trained model does not exist, training SinGAN for SR')
                Gs, Zs, reals, NoiseAmp = functions.load_trained_pyramid(opt)
                opt.mode = 'paint2image'
            else:
                train_paint(opt, Gs, Zs, reals, NoiseAmp, centers, opt.paint_start_scale)
                opt.mode = 'paint2image'
        out = SinGAN_generate(Gs[n:], Zs[n:], reals, NoiseAmp[n:], opt, in_s, n=n, num_samples=1)
        plt.imsave('%s/start_scale=%d.png' % (dir2save, opt.paint_start_scale), functions.convert_image_np(out.detach()), vmin=0, vmax=1)

X shape= (1562, 2090, 3)
x after imread=== (1562, 2090, 3)


In [13]:
#plt.imshow(real.cpu().numpy())

TypeError: Invalid shape (1, 3, 187, 250) for image data

In [15]:
in_s.shape

torch.Size([1, 3, 33, 43])